In [ ]:
!nvidia-smi

Thu Nov 28 18:42:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Fine-Tuning Pegasus Model for Abstractive Text Summarization

This script fine-tunes a pre-trained Pegasus model for abstractive text summarization using the CNN/Daily Mail dataset. The process includes data loading, model preparation, training, evaluation, and saving the best model based on validation loss.

## Key Features
- **Model**: Uses the 'google/pegasus-large' pre-trained model.
- **Dataset**: Trains on a subset of the CNN/Daily Mail dataset (1000 samples).
- **Epochs**: Fine-tunes for a specified number of epochs.
- **Metrics**: Evaluates performance using BLEU and ROUGE scores.
- **Batch Processing**: Implements gradient accumulation to handle larger batch sizes.
- **Model Saving**: Saves the best performing model based on validation loss.
- **Comparison**: Compares the performance of the base model and the fine-tuned model.
- **Performance Improvement**: Reports the improvement in BLEU and ROUGE scores achieved through fine-tuning.

## Performance Metrics

### Base Model Performance
- **BLEU**: 0.026
- **ROUGE-1**: 0.299
- **ROUGE-2**: 0.098
- **ROUGE-L**: 0.192

### Fine-Tuned Model Performance
- **BLEU Score**: 0.05830824748912286
- **ROUGE-1**: 0.33447035562023225
- **ROUGE-2**: 0.13160724821103553
- **ROUGE-L**: 0.2448122833529142

### Performance Improvement
- **BLEU**: +0.032019698130967805
- **ROUGE-1**: +0.035372132340698026
- **ROUGE-2**: +0.033823160043814124
- **ROUGE-L**: +0.052479002091481874

In [ ]:
!pip install datasets
!pip install rouge_score
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fedca527489663b672b233399

In [ ]:
import random
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from transformers import get_linear_schedule_with_warmup
import os

# Set random seed for reproducibility
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [ ]:
# Load pre-trained model and tokenizer
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
# Count the number of trainable parameters in the model
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {total_params:,}")

Total Trainable Parameters: 568,699,904


In [ ]:
# Constants
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 5e-5

In [ ]:
# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Prepare data
full_train_data = dataset["train"].select(range(1000))  # 1000
train_size = int(0.9 * len(full_train_data))
val_size = len(full_train_data) - train_size

train_data = full_train_data.select(range(train_size))
val_data = full_train_data.select(range(train_size, len(full_train_data)))

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
print("Length of train_data:", len(train_data))
print("Length of val_data:", len(val_data))

Length of train_data: 900
Length of val_data: 100


In [ ]:
class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=MAX_INPUT_LENGTH, max_target_length=MAX_TARGET_LENGTH):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data[idx]["article"]
        summary = self.data[idx]["highlights"]

        inputs = self.tokenizer(article, max_length=self.max_input_length, truncation=True, padding="max_length", return_tensors="pt")
        targets = self.tokenizer(summary, max_length=self.max_target_length, truncation=True, padding="max_length", return_tensors="pt")

        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": targets.input_ids.squeeze()
        }

train_dataset = SummarizationDataset(train_data, tokenizer)
val_dataset = SummarizationDataset(val_data, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def evaluate(model, data_loader):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_TARGET_LENGTH)

            for i in range(len(input_ids)):
                reference = tokenizer.decode(labels[i], skip_special_tokens=True)
                generated_summary = tokenizer.decode(generated_ids[i], skip_special_tokens=True)

                bleu_score = sentence_bleu([reference.split()], generated_summary.split())
                bleu_scores.append(bleu_score)

                rouge_result = scorer.score(reference, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(rouge_result[metric].fmeasure)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = {metric: sum(scores) / len(scores) for metric, scores in rouge_scores.items()}

    return avg_bleu, avg_rouge

# Evaluate base model
print("Evaluating base model...")
base_bleu, base_rouge = evaluate(model, val_loader)
print("Base Model Performance:")
print(f"BLEU Score: {base_bleu}")
print(f"ROUGE Scores: {base_rouge}")

Evaluating base model...


Evaluating: 100%|██████████| 25/25 [01:22<00:00,  3.30s/it]

Base Model Performance:
BLEU Score: 0.026288549358155056
ROUGE Scores: {'rouge1': 0.2990982232795342, 'rouge2': 0.09778408816722141, 'rougeL': 0.19233328126143232}


In [ ]:
# Fine-tuning
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

best_val_loss = float('inf')
best_model_path = 'best_pegasus_model.pth'

for epoch in range(NUM_EPOCHS):
    model.train()
    total_train_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")

    for i, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss = loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()

        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        progress_bar.set_postfix({"train_loss": loss.item() * GRADIENT_ACCUMULATION_STEPS})

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # Evaluate every 2 epochs
    if (epoch + 1) % 2 == 0:
        print(f"Evaluating after epoch {epoch+1}...")
        current_bleu, current_rouge = evaluate(model, val_loader)
        print(f"Current BLEU Score: {current_bleu}")
        print(f"Current ROUGE Scores: {current_rouge}")

    # Save the model if it's the best so far based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation loss: {best_val_loss:.4f}")

# Load the best model for final evaluation
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))
    print(f"Loaded best model from {best_model_path}")
else:
    print("No saved model found. Using the model from the last epoch.")

# Evaluate fine-tuned model
print("Evaluating fine-tuned model...")
fine_tuned_bleu, fine_tuned_rouge = evaluate(model, val_loader)
print("Fine-tuned Model Performance:")
print(f"BLEU Score: {fine_tuned_bleu}")
print(f"ROUGE Scores: {fine_tuned_rouge}")

# Print performance improvement
print("Performance Improvement:")
print(f"BLEU: {fine_tuned_bleu - base_bleu}")
print(f"ROUGE-1: {fine_tuned_rouge['rouge1'] - base_rouge['rouge1']}")
print(f"ROUGE-2: {fine_tuned_rouge['rouge2'] - base_rouge['rouge2']}")
print(f"ROUGE-L: {fine_tuned_rouge['rougeL'] - base_rouge['rougeL']}")

Epoch 1/10 [Val]: 100%|██████████| 25/25 [00:05<00:00,  4.65it/s]


Epoch 1/10, Train Loss: 7.4636, Val Loss: 6.8137
New best model saved with validation loss: 6.8137


Epoch 2/10 [Val]: 100%|██████████| 25/25 [00:05<00:00,  4.65it/s]


Epoch 2/10, Train Loss: 6.7187, Val Loss: 6.3782
Evaluating after epoch 2...


Evaluating:   4%|▍         | 1/25 [00:03<01:18,  3.26s/it]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Evaluating:   8%|▊         | 2/25 [00:06<01:14,  3.23s/it]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram

...

Current BLEU Score: 0.05830824748912286
Current ROUGE Scores: {'rouge1': 0.33447035562023225, 'rouge2': 0.13160724821103553, 'rougeL': 0.2448122833529142}
New best model saved with validation loss: 0.8053


<ipython-input-35-842a4292022c>:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Loaded best model from best_pegasus_model.pth
Evaluating fine-tuned model...


Evaluating: 100%|██████████| 25/25 [01:02<00:00,  2.51s/it]

Fine-tuned Model Performance:
BLEU Score: 0.05830824748912286
ROUGE Scores: {'rouge1': 0.33447035562023225, 'rouge2': 0.13160724821103553, 'rougeL': 0.2448122833529142}
Performance Improvement:
BLEU: 0.032019698130967805
ROUGE-1: 0.035372132340698026
ROUGE-2: 0.033823160043814124
ROUGE-L: 0.052479002091481874


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def generate_summary(model, article, tokenizer, max_length=128):
    inputs = tokenizer(article, max_length=1024, truncation=True, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Generate summaries with base model
print("Base Model Summaries:")
for i in range(3):  # Generate 3 summaries
    article = val_data[i]["article"]
    reference = val_data[i]["highlights"]
    generated = generate_summary(model, article, tokenizer)
    print(f"\nArticle {i+1}:")
    print(f"Reference: {reference}")
    print(f"Generated: {generated}")
    print("-" * 50)

Base Model Summaries:

Article 1:
Reference: Photos of Taliban in the uniforms of dead French soldiers provokes outrage .
Magazine Paris Match features photos of Taliban and their commander .
10 French troops were killed and a further 21 injured in an ambush .
Generated: Joel Le Pahun, father of one of the killed soldiers, told the newspaper the pictures were "despicable." Green MP Daniel Cohn-Bendit called them "voyeurism." However, Paris Match editor Laurent Valdiguie defended the publication, saying it was "legitimate" given the importance of the story.
--------------------------------------------------

Article 2:
Reference: The Middle East's richest man is Prince Alwaleed Bin Talal Alsaud .
He ranks 19th in the world in the Forbes Rich List .
Seven other billionaires from the Middle East rank in the top 100 .
Generated: The Middle East's richest man: Prince Alwaleed Bin Talal Alsaud . The Middle East's richest man is Prince Alwaleed Bin Talal Alsaud, the 51 year old Saudi who has 

In [ ]:
# Load the fine-tuned model
best_model_path = "/content/drive/MyDrive/NLP-Project/best_pegasus_model.pth"
fine_tuned_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
fine_tuned_model.load_state_dict(torch.load(best_model_path))
fine_tuned_model.to(device)

print("\nFine-tuned Model Summaries:")
for i in range(3):  # Generate 3 summaries
    article = val_data[i]["article"]
    reference = val_data[i]["highlights"]
    generated = generate_summary(fine_tuned_model, article, tokenizer)
    print(f"\nArticle {i+1}:")
    print(f"Reference: {reference}")
    print(f"Generated: {generated}")
    print("-" * 50)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-601908a927f4>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `t


Fine-tuned Model Summaries:

Article 1:
Reference: Photos of Taliban in the uniforms of dead French soldiers provokes outrage .
Magazine Paris Match features photos of Taliban and their commander .
10 French troops were killed and a further 21 injured in an ambush .
Generated: Paris Match includes photos of Taliban fighters and their commander . The latest edition includes photos of the Taliban fighters and their commander, "Farouki," wearing French uniforms . Father of one of the 10 French soldiers says pictures are "despicable"
--------------------------------------------------

Article 2:
Reference: The Middle East's richest man is Prince Alwaleed Bin Talal Alsaud .
He ranks 19th in the world in the Forbes Rich List .
Seven other billionaires from the Middle East rank in the top 100 .
Generated: Prince Alwaleed Bin Talal Alsaud ranks 19th in the list and is considered to be the most active and successful investor in the Middle East . He took his investment vehicle, Kingdom Holding,

# Fine-Tuning Pegasus Model for Abstractive Text Summarization on datasize of 5000 articles.

This script fine-tunes a pre-trained Pegasus model for abstractive text summarization using the CNN/Daily Mail dataset. The process includes data loading, model preparation, training, evaluation, and saving the best model based on validation loss.

BLEU Score: 0.07084515511453165
ROUGE Scores: {'rouge1': 0.3628623757565976, 'rouge2': 0.14441779133424118, 'rougeL': 0.25522549668218547}


## Performance Metrics

### Base Model Performance
- **BLEU**: 0.027228885603706635
- **ROUGE-1**: 0.027228885603706635
- **ROUGE-2**: 0.027228885603706635
- **ROUGE-L**: 0.027228885603706635

### Fine-Tuned Model Performance
- **BLEU Score**: 0.07084515511453165
- **ROUGE-1**: 0.3628623757565976
- **ROUGE-2**: 0.14441779133424118
- **ROUGE-L**: 0.25522549668218547



In [ ]:
import random
import time
import os
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from google.colab import drive, files

# Mount Google Drive
drive.mount('/content/drive')

# Install required libraries
!pip install transformers datasets rouge_score nltk tqdm matplotlib

# Constants
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 5e-5

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_data(num_samples=1000):
    dataset = load_dataset("cnn_dailymail", "3.0.0")
    full_train_data = dataset["train"].select(range(num_samples))
    train_size = int(0.9 * len(full_train_data))
    train_data = full_train_data.select(range(train_size))
    val_data = full_train_data.select(range(train_size, len(full_train_data)))
    return train_data, val_data

class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=MAX_INPUT_LENGTH, max_target_length=MAX_TARGET_LENGTH):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data[idx]["article"]
        summary = self.data[idx]["highlights"]
        inputs = self.tokenizer(article, max_length=self.max_input_length, truncation=True, padding="max_length", return_tensors="pt")
        targets = self.tokenizer(summary, max_length=self.max_target_length, truncation=True, padding="max_length", return_tensors="pt")
        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": targets.input_ids.squeeze()
        }

def create_dataloaders(train_data, val_data, tokenizer):
    train_dataset = SummarizationDataset(train_data, tokenizer)
    val_dataset = SummarizationDataset(val_data, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, val_loader

def evaluate(model, data_loader, tokenizer, device):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_TARGET_LENGTH)

            for i in range(len(input_ids)):
                reference = tokenizer.decode(labels[i], skip_special_tokens=True)
                generated_summary = tokenizer.decode(generated_ids[i], skip_special_tokens=True)

                bleu_score = sentence_bleu([reference.split()], generated_summary.split())
                bleu_scores.append(bleu_score)

                rouge_result = scorer.score(reference, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(rouge_result[metric].fmeasure)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = {metric: sum(scores) / len(scores) for metric, scores in rouge_scores.items()}

    return avg_bleu, avg_rouge

def train_model(model, train_loader, val_loader, tokenizer, device, num_epochs):
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    num_training_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    best_val_loss = float('inf')
    best_model_path = '/content/drive/My Drive/NLP-Project/best_pegasus_model_modular_script_test.pth'

    train_losses = []
    val_losses = []
    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for i, batch in enumerate(progress_bar):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()

            loss = loss / GRADIENT_ACCUMULATION_STEPS
            loss.backward()

            if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            progress_bar.set_postfix({"train_loss": loss.item() * GRADIENT_ACCUMULATION_STEPS})

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Evaluate every 2 epochs
        if (epoch + 1) % 2 == 0:
            print(f"Evaluating after epoch {epoch+1}...")
            current_bleu, current_rouge = evaluate(model, val_loader, tokenizer, device)
            bleu_scores.append(current_bleu)
            for metric in rouge_scores:
                rouge_scores[metric].append(current_rouge[metric])
            print(f"Current BLEU Score: {current_bleu}")
            print(f"Current ROUGE Scores: {current_rouge}")

        # Save the model if it's the best so far based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total training time: {training_time:.2f} seconds")

    return train_losses, val_losses, bleu_scores, rouge_scores, training_time

def plot_training_progress(train_losses, val_losses, bleu_scores, rouge_scores):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 2)
    plt.plot(bleu_scores)
    plt.title('BLEU Score')

    plt.subplot(2, 2, 3)
    for metric, scores in rouge_scores.items():
        plt.plot(scores, label=metric)
    plt.legend()
    plt.title('ROUGE Scores')

    plt.tight_layout()
    plt.savefig('training_progress.png')
    plt.close()

    # Download the plot
    files.download('training_progress.png')

def generate_summary(model, article, tokenizer, device, max_length=128):
    inputs = tokenizer(article, max_length=1024, truncation=True, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def main():
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
    model.to(device)

    train_data, val_data = load_data(num_samples=20000)
    train_loader, val_loader = create_dataloaders(train_data, val_data, tokenizer)

    print("Evaluating base model...")
    base_bleu, base_rouge = evaluate(model, val_loader, tokenizer, device)
    print("Base Model Performance:")
    print(f"BLEU Score: {base_bleu}")
    print(f"ROUGE Scores: {base_rouge}")

    train_losses, val_losses, bleu_scores, rouge_scores, training_time = train_model(model, train_loader, val_loader, tokenizer, device, NUM_EPOCHS)

    plot_training_progress(train_losses, val_losses, bleu_scores, rouge_scores)

    print("Evaluating fine-tuned model...")
    fine_tuned_bleu, fine_tuned_rouge = evaluate(model, val_loader, tokenizer, device)
    print("Fine-tuned Model Performance:")
    print(f"BLEU Score: {fine_tuned_bleu}")
    print(f"ROUGE Scores: {fine_tuned_rouge}")

    print("Performance Improvement:")
    print(f"BLEU: {fine_tuned_bleu - base_bleu}")
    print(f"ROUGE-1: {fine_tuned_rouge['rouge1'] - base_rouge['rouge1']}")
    print(f"ROUGE-2: {fine_tuned_rouge['rouge2'] - base_rouge['rouge2']}")
    print(f"ROUGE-L: {fine_tuned_rouge['rougeL'] - base_rouge['rougeL']}")

    print(f"Total training time: {training_time:.2f} seconds")

    # Generate example summaries
    print("\nGenerating example summaries...")
    for i in range(3):
        article = val_data[i]["article"]
        reference = val_data[i]["highlights"]
        generated = generate_summary(model, article, tokenizer, device)
        print(f"\nArticle {i+1}:")
        print(f"Reference: {reference}")
        print(f"Generated: {generated}")
        print("-" * 50)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating base model...


Evaluating:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Evaluating:   4%|▍         | 5/125 [00:16<06:28,  3.23s/it]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overl

Base Model Performance:
BLEU Score: 0.027228885603706635
ROUGE Scores: {'rouge1': 0.29546262944841484, 'rouge2': 0.09988257187753315, 'rougeL': 0.1893963564402305}


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluating fine-tuned model...


Evaluating: 100%|██████████| 125/125 [04:33<00:00,  2.19s/it]


Fine-tuned Model Performance:
BLEU Score: 0.07084515511453165
ROUGE Scores: {'rouge1': 0.3628623757565976, 'rouge2': 0.14441779133424118, 'rougeL': 0.25522549668218547}
Performance Improvement:
BLEU: 0.04361626951082502
ROUGE-1: 0.06739974630818274
ROUGE-2: 0.04453521945670803
ROUGE-L: 0.06582914024195496
Total training time: 8732.52 seconds

Generating example summaries...

Article 1:
Reference: Belgian architect imagines climate refugees living on a futuristic Lilypad ecopolis .
The structure would support 50,000 inhabitants in a zero carbon environment .
The goal is to "create a harmonious coexistence of humans and nature"
Generated: The Lilypad is the creation of Belgian architect Vincent Callebaut . "It is" he says, "a true amphibian, half aquatic and half terrestrial city, able to accommodate 50,000 inhabitants"
--------------------------------------------------

Article 2:
Reference: Swedish entrepreneur to open a Jumbo Hostel at Arlanda airport in Sweden .
Decommissioned Boeing

## Fine-Tuning Pegasus Model for Abstractive Text Summarization on datasize of 20,000 articles.

### Pegasus Model Training Results: CNN-Daily Mail Dataset

#### Training Overview
Model training was conducted over 10 epochs with the following specifications:
- **Base Model**: google/pegasus-large
- **Dataset**: CNN-Daily Mail
- **Batch Size**: 4
- **Initial Learning Rate**: 2e-5

---

#### Training Metrics Progression
| Epoch | Train Loss | Val Loss | Learning Rate | BLEU  | ROUGE-1 | ROUGE-2 | ROUGE-L |
|-------|------------|----------|---------------|-------|---------|---------|---------|
| 1     | 7.9441     | 6.6767   | 5.00e-06      | -     | -       | -       | -       |
| 2     | 6.2990     | 4.5098   | 9.99e-06      | 0.0308| 0.1535  | 0.0653  | 0.1063  |
| 3     | 2.1489     | 0.8050   | 1.50e-05      | -     | -       | -       | -       |
| 4     | 0.8219     | 0.7685   | 2.00e-05      | 0.0811| 0.3553  | 0.1535  | 0.2530  |
| 5     | 0.7729     | 0.7573   | 1.94e-05      | -     | -       | -       | -       |
| 6     | 0.7418     | 0.7504   | 1.89e-05      | 0.0827| 0.3587  | 0.1552  | 0.2544  |
| 7     | 0.7193     | 0.7476   | 1.83e-05      | -     | -       | -       | -       |
| 8     | 0.7001     | 0.7442   | 1.78e-05      | 0.0815| 0.3591  | 0.1542  | 0.2572  |
| 9     | 0.6832     | 0.7425   | 1.72e-05      | -     | -       | -       | -       |
| 10    | 0.6684     | 0.7425   | 1.67e-05      | 0.0843| 0.3626  | 0.1569  | 0.2585  |

---

#### Base Model Performance
- **BLEU**: 0.027228885603706635
- **ROUGE-1**: 0.027228885603706635
- **ROUGE-2**: 0.027228885603706635
- **ROUGE-L**: 0.027228885603706635

#### Final Model Performance
- **Best Validation Loss**: 0.7425 (Epoch 10)
- **Final BLEU**: 0.0843
- **Final ROUGE Scores**:
  - ROUGE-1: 0.3626
  - ROUGE-2: 0.1569
  - ROUGE-L: 0.2585
- **Total Training Time**: 17,830.88 seconds (~4.95 hours)

This training report highlights the progress and outcomes of fine-tuning Pegasus on the CNN-Daily Mail dataset. Further optimizations and improvements can build upon these results for better performance.

In [ ]:
! pip install transformers datasets rouge_score nltk tqdm matplotlib
import random
import time
import os
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from google.colab import drive, files

# Mount Google Drive
drive.mount('/content/drive')

# Install required libraries
!pip install transformers datasets rouge_score nltk tqdm matplotlib

# Modified constants
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 2e-5  # Reduced from 5e-5
WARMUP_RATIO = 0.1  # Added warmup
WEIGHT_DECAY = 0.01  # Added weight decay

def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_data(num_samples=1000):
    dataset = load_dataset("cnn_dailymail", "3.0.0")
    full_train_data = dataset["train"].select(range(num_samples))
    train_size = int(0.9 * len(full_train_data))
    train_data = full_train_data.select(range(train_size))
    val_data = full_train_data.select(range(train_size, len(full_train_data)))
    return train_data, val_data

class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length=MAX_INPUT_LENGTH, max_target_length=MAX_TARGET_LENGTH):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data[idx]["article"]
        summary = self.data[idx]["highlights"]
        inputs = self.tokenizer(article, max_length=self.max_input_length, truncation=True, padding="max_length", return_tensors="pt")
        targets = self.tokenizer(summary, max_length=self.max_target_length, truncation=True, padding="max_length", return_tensors="pt")
        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": targets.input_ids.squeeze()
        }

def create_dataloaders(train_data, val_data, tokenizer):
    train_dataset = SummarizationDataset(train_data, tokenizer)
    val_dataset = SummarizationDataset(val_data, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, val_loader

def evaluate(model, data_loader, tokenizer, device):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=MAX_TARGET_LENGTH)

            for i in range(len(input_ids)):
                reference = tokenizer.decode(labels[i], skip_special_tokens=True)
                generated_summary = tokenizer.decode(generated_ids[i], skip_special_tokens=True)

                bleu_score = sentence_bleu([reference.split()], generated_summary.split())
                bleu_scores.append(bleu_score)

                rouge_result = scorer.score(reference, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(rouge_result[metric].fmeasure)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = {metric: sum(scores) / len(scores) for metric, scores in rouge_scores.items()}

    return avg_bleu, avg_rouge

def train_model(model, train_loader, val_loader, tokenizer, device, num_epochs):
    # Modified optimizer with weight decay
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": WEIGHT_DECAY,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)

    # Modified scheduler with warmup
    num_training_steps = len(train_loader) * num_epochs
    num_warmup_steps = int(num_training_steps * WARMUP_RATIO)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    best_val_loss = float('inf')
    best_model_path = '/content/drive/My Drive/NLP-Project/best_pegasus_model_20000_data.pth'
    patience = 3  # Early stopping patience
    no_improve = 0

    train_losses = []
    val_losses = []
    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        # Training loop with gradient clipping
        for i, batch in enumerate(progress_bar):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()

            loss = loss / GRADIENT_ACCUMULATION_STEPS
            loss.backward()

            if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                # Add gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            progress_bar.set_postfix({"train_loss": loss.item() * GRADIENT_ACCUMULATION_STEPS})

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation with early stopping
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Average Train Loss: {avg_train_loss:.4f}")
        print(f"Average Val Loss: {avg_val_loss:.4f}")
        print(f"Learning Rate: {scheduler.get_last_lr()[0]:.2e}")


        # Evaluate every 2 epochs
        if (epoch + 1) % 2 == 0:
            print(f"Evaluating after epoch {epoch+1}...")
            current_bleu, current_rouge = evaluate(model, val_loader, tokenizer, device)
            bleu_scores.append(current_bleu)
            for metric in rouge_scores:
                rouge_scores[metric].append(current_rouge[metric])
            print(f"Current BLEU Score: {current_bleu}")
            print(f"Current ROUGE Scores: {current_rouge}")

        # Save best model and check for early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with validation loss: {best_val_loss:.4f}")
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break




    end_time = time.time()
    training_time = end_time - start_time

    return train_losses, val_losses, bleu_scores, rouge_scores, training_time

def plot_training_progress(train_losses, val_losses, bleu_scores, rouge_scores):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 2)
    plt.plot(bleu_scores)
    plt.title('BLEU Score')

    plt.subplot(2, 2, 3)
    for metric, scores in rouge_scores.items():
        plt.plot(scores, label=metric)
    plt.legend()
    plt.title('ROUGE Scores')

    plt.tight_layout()
    plt.savefig('training_progress.png')
    plt.close()

    # Download the plot
    files.download('training_progress.png')

def generate_summary(model, article, tokenizer, device, max_length=128):
    inputs = tokenizer(article, max_length=1024, truncation=True, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def main():
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
    model.to(device)

    train_data, val_data = load_data(num_samples=10000)
    train_loader, val_loader = create_dataloaders(train_data, val_data, tokenizer)

    print("Evaluating base model...")
    base_bleu, base_rouge = evaluate(model, val_loader, tokenizer, device)
    print("Base Model Performance:")
    print(f"BLEU Score: {base_bleu}")
    print(f"ROUGE Scores: {base_rouge}")

    train_losses, val_losses, bleu_scores, rouge_scores, training_time = train_model(model, train_loader, val_loader, tokenizer, device, NUM_EPOCHS)

    plot_training_progress(train_losses, val_losses, bleu_scores, rouge_scores)

    print("Evaluating fine-tuned model...")
    fine_tuned_bleu, fine_tuned_rouge = evaluate(model, val_loader, tokenizer, device)
    print("Fine-tuned Model Performance:")
    print(f"BLEU Score: {fine_tuned_bleu}")
    print(f"ROUGE Scores: {fine_tuned_rouge}")

    print("Performance Improvement:")
    print(f"BLEU: {fine_tuned_bleu - base_bleu}")
    print(f"ROUGE-1: {fine_tuned_rouge['rouge1'] - base_rouge['rouge1']}")
    print(f"ROUGE-2: {fine_tuned_rouge['rouge2'] - base_rouge['rouge2']}")
    print(f"ROUGE-L: {fine_tuned_rouge['rougeL'] - base_rouge['rougeL']}")

    print(f"Total training time: {training_time:.2f} seconds")

    # Generate example summaries
    print("\nGenerating example summaries...")
    for i in range(3):
        article = val_data[i]["article"]
        reference = val_data[i]["highlights"]
        generated = generate_summary(model, article, tokenizer, device)
        print(f"\nArticle {i+1}:")
        print(f"Reference: {reference}")
        print(f"Generated: {generated}")
        print("-" * 50)

if __name__ == "__main__":
    main()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=38652ba2407d7d08193697575e8fecd4e01c3e851ab0b4612c7ccfeca9df7058
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that a

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluating fine-tuned model...


Evaluating: 100%|██████████| 250/250 [09:58<00:00,  2.40s/it]


Fine-tuned Model Performance:
BLEU Score: 0.08428911517596681
ROUGE Scores: {'rouge1': 0.36259173481518503, 'rouge2': 0.15686449208025968, 'rougeL': 0.2584988971139973}
Total training time: 17830.88 seconds

Generating example summaries...

Article 1:
Reference: Maj. Curtis Daniel Miller of Palacios, Texas, was shot down on March 29, 1972 .
Miller and his crew were flying over southern Laos when a missile struck their plane .
Miller will be buried with full military honors at the Dallas-Ft. Worth National Cemetery .
Generated: Air Force Maj. Curtis Daniel Miller of Palacios, Texas, was one of 14 men . Miller and his crew were flying over southern Laos when a missile struck their plane . Rescue teams had to call off the search after two days because of heavy fighting .
--------------------------------------------------

Article 2:
Reference: Group files motion for preliminary injunction against a Mississippi school district .
Case will be argued before a federal judge in Mississippi on 

In [6]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


# Training a Scaled-Down Pegasus Model From Scratch

## Model Architecture & Dataset
- Created a smaller Pegasus model with:
  - 8 encoder and decoder layers (reduced from 16)
  - 512 dimensional embeddings (reduced from 1024)
  - 2048 FFN dimensions (reduced from 4096)
  - ~108.5M total parameters, with 108M trainable parameters

- Dataset: CNN/DailyMail
  - Training samples: 13,500
  - Validation samples: 1,500
  - Max input length: 512 tokens
  - Max output length: 128 tokens

## Training Configuration
- Batch size: 4 with gradient accumulation steps of 2
- Initial learning rate: 2e-5 with linear warmup
- ReduceLROnPlateau scheduler with factor 0.1
- Weight decay: 0.01
- Training epochs: 20
- Early stopping patience: 3

## Training Results
- Loss progression:
  - Initial training loss: 10.29 → Final training loss: 2.13
  - Initial validation loss: 8.31 → Final validation loss: 2.16
  - Consistent decrease in both training and validation loss

- ROUGE Scores progression:
  - ROUGE-1: 0.0 → 0.122 (12.2%)
  - ROUGE-2: 0.0 → 0.014 (1.4%)
  - ROUGE-L: 0.0 → 0.101 (10.1%)

## Key Observations
1. Model showed steady improvement in both training and validation loss
2. ROUGE scores improved gradually but remained relatively low
3. Training completed in ~5 hours (~17,975 seconds)
4. No significant overfitting observed (validation loss tracks training loss)
5. Memory usage remained stable (~4.6-4.7 GB GPU memory)

## Limitations & Future Work
1. Limited performance on ROUGE metrics suggests room for improvement
2. Small batch size due to memory constraints
3. Could benefit from:
   - Larger model capacity
   - More training data
   - Longer training time
   - Larger batch size with more computational resources

In [2]:
!pip install transformers datasets rouge_score nltk tqdm matplotlib

import os
import torch
from transformers import PegasusConfig, PegasusForConditionalGeneration, PegasusTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from google.colab import drive, files

# Mount Google Drive
drive.mount('/content/drive')

def verify_model_size(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

def cleanup():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    plt.close('all')

def create_small_pegasus_config():
    '''Create a smaller Pegasus configuration suitable for training on 3000 samples.'''
    config = PegasusConfig(
        vocab_size=96103,  # Original vocab size for tokenizer compatibility
        encoder_layers=8,  # Reduced from 16
        decoder_layers=8,  # Reduced from 16
        encoder_attention_heads=16,  # Reduced from 16
        decoder_attention_heads=16,  # Reduced from 16
        encoder_ffn_dim=2048,  # Reduced from 4096
        decoder_ffn_dim=2048,  # Reduced from 4096
        d_model=512,  # Reduced from 1024
        max_position_embeddings=512,  # Reduced context length
        pad_token_id=0,
        eos_token_id=1,
        forced_eos_token_id=1,
        activation_function='gelu',
        dropout=0.2,  # Increased dropout for smaller dataset
        attention_dropout=0.2,
        activation_dropout=0.2,
        num_beams=4,
        encoder_layerdrop=0.1,  # Added layerdrop for regularization
        decoder_layerdrop=0.1,
        scale_embedding=True,
        use_cache=True,
        is_encoder_decoder=True
    )
    return config

TRAINING_PARAMS = {
    'MAX_INPUT_LENGTH': 512,
    'MAX_TARGET_LENGTH': 128,
    'BATCH_SIZE': 4,
    'GRADIENT_ACCUMULATION_STEPS': 2,
    'NUM_EPOCHS': 20,
    'LEARNING_RATE': 2e-5,
    'WARMUP_RATIO': 0.1,
    'WEIGHT_DECAY': 0.01,
    'EARLY_STOPPING_PATIENCE': 3
}

def set_seed(seed=42):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# def load_data(num_samples=3000):
#     dataset = load_dataset('cnn_dailymail', '3.0.0')
#     full_train_data = dataset['train'].select(range(num_samples))
#     train_size = int(0.9 * len(full_train_data))
#     train_data = full_train_data.select(range(train_size))
#     val_data = full_train_data.select(range(train_size, len(full_train_data)))
#     return train_data, val_data

def load_data(num_samples=1000):
    # Load the dataset
    dataset = load_dataset('cnn_dailymail', '3.0.0')

    # Calculate how many samples we want for each split
    train_samples = int(0.9 * num_samples)  # 90% of samples for training
    val_samples = num_samples - train_samples  # 10% of samples for validation

    # Randomly select indices for training and validation
    train_indices = range(train_samples)
    val_indices = range(len(dataset['validation']))[:val_samples]

    # Select the samples from the respective splits
    train_data = dataset['train'].select(train_indices)
    val_data = dataset['validation'].select(val_indices)

    print(f"Training samples: {len(train_data)}")
    print(f"Validation samples: {len(val_data)}")

    return train_data, val_data



class SummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_input_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_input_length = max_input_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        article = self.data[idx]['article']
        summary = self.data[idx]['highlights']
        inputs = self.tokenizer(article, max_length=self.max_input_length, truncation=True, padding='max_length', return_tensors='pt')
        targets = self.tokenizer(summary, max_length=self.max_target_length, truncation=True, padding='max_length', return_tensors='pt')
        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': targets.input_ids.squeeze()
        }

def create_dataloaders(train_data, val_data, tokenizer, batch_size):
    train_dataset = SummarizationDataset(train_data, tokenizer, TRAINING_PARAMS['MAX_INPUT_LENGTH'], TRAINING_PARAMS['MAX_TARGET_LENGTH'])
    val_dataset = SummarizationDataset(val_data, tokenizer, TRAINING_PARAMS['MAX_INPUT_LENGTH'], TRAINING_PARAMS['MAX_TARGET_LENGTH'])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

def evaluate(model, data_loader, tokenizer, device):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=TRAINING_PARAMS['MAX_TARGET_LENGTH'])

            for i in range(len(input_ids)):
                reference = tokenizer.decode(labels[i], skip_special_tokens=True)
                generated_summary = tokenizer.decode(generated_ids[i], skip_special_tokens=True)

                rouge_result = scorer.score(reference, generated_summary)
                for metric in rouge_scores:
                    rouge_scores[metric].append(rouge_result[metric].fmeasure)

    avg_rouge = {metric: sum(scores) / len(scores) for metric, scores in rouge_scores.items()}

    return avg_rouge



def train_model(model, train_loader, val_loader, tokenizer, device, num_epochs):
    try:
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': TRAINING_PARAMS['WEIGHT_DECAY'],
            },
            {
                'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
            },
        ]
        optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=TRAINING_PARAMS['LEARNING_RATE'])
        num_training_steps = len(train_loader) * num_epochs
        num_warmup_steps = int(num_training_steps * TRAINING_PARAMS['WARMUP_RATIO'])
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)
        # Scheduler for reducing learning rate when validation loss stagnates
        scheduler_plateau = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.1,
            patience=4,
            verbose=True
        )
        best_val_loss = float('inf')
        best_model_path = "/content/drive/My Drive/NLP-Project/best_pegasus_scartch.pt"
        patience = TRAINING_PARAMS['EARLY_STOPPING_PATIENCE']
        no_improve = 0

        train_losses = []
        val_losses = []
        rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

        start_time = time.time()

        for epoch in range(num_epochs):
            # Initialize these at the start of each epoch
            total_train_loss = 0
            batch_count = 0
            avg_train_loss = float('inf')  # Default value

            try:
                # Clear GPU cache before each epoch
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    print(f"GPU Memory before epoch {epoch + 1}: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

                model.train()
                progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

                for i, batch in enumerate(progress_bar):
                    try:
                        input_ids = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
                        labels = batch['labels'].to(device)

                        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                        current_loss = outputs.loss / TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS']
                        current_loss.backward()

                        if (i + 1) % TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS'] == 0:
                            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                            optimizer.step()
                            scheduler.step()
                            optimizer.zero_grad()

                        # Update loss tracking
                        total_train_loss += current_loss.item() * TRAINING_PARAMS['GRADIENT_ACCUMULATION_STEPS']
                        batch_count += 1

                        # Update progress bar
                        if batch_count > 0:
                            avg_train_loss = total_train_loss / batch_count
                            progress_bar.set_postfix({'loss': avg_train_loss})

                        # Clear memory after optimization step
                        del outputs, current_loss
                        if torch.cuda.is_available():
                            torch.cuda.empty_cache()

                    except RuntimeError as e:
                        if "out of memory" in str(e):
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()
                            print(f"WARNING: out of memory in batch {i}. Skipping batch...")
                            print(f"GPU Memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
                            continue
                        else:
                            raise e

                if batch_count > 0:
                    avg_train_loss = total_train_loss / batch_count
                    train_losses.append(avg_train_loss)

                # Validation phase
                model.eval()
                total_val_loss = 0
                val_batch_count = 0

                with torch.no_grad():
                    for batch in tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]'):
                        try:
                            input_ids = batch['input_ids'].to(device)
                            attention_mask = batch['attention_mask'].to(device)
                            labels = batch['labels'].to(device)

                            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                            total_val_loss += outputs.loss.item()
                            val_batch_count += 1

                            # Clear memory after each validation batch
                            del outputs
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()

                        except RuntimeError as e:
                            if "out of memory" in str(e):
                                if torch.cuda.is_available():
                                    torch.cuda.empty_cache()
                                print("WARNING: out of memory in validation. Skipping batch...")
                                continue
                            else:
                                raise e

                if val_batch_count > 0:
                    avg_val_loss = total_val_loss / val_batch_count
                    val_losses.append(avg_val_loss)

                    print(f"Epoch {epoch+1}/{num_epochs}")
                    print(f"Average Train Loss: {avg_train_loss:.4f}")
                    print(f"Average Val Loss: {avg_val_loss:.4f}")
                    print(f"Learning Rate: {scheduler.get_last_lr()[0]:.2e}")


                    scheduler_plateau.step(avg_val_loss)
                    current_lr = optimizer.param_groups[0]['lr']
                    print(f"Updated learning rate after plateau adjustment: {current_lr}")

                    # # Evaluate ROUGE scores
                    # if (epoch + 1) % 2 == 0:
                    print(f"Evaluating after epoch {epoch+1}...")
                    current_rouge = evaluate(model, val_loader, tokenizer, device)
                    for metric in rouge_scores:
                        rouge_scores[metric].append(current_rouge[metric])

                    print(f"Current ROUGE Scores: {current_rouge}")

                    # Model saving with error handling
                    if avg_val_loss < best_val_loss:
                        best_val_loss = avg_val_loss
                        try:
                            torch.save(model.state_dict(), best_model_path)
                            print(f"New best model saved with validation loss: {best_val_loss:.4f}")
                        except Exception as e:
                            print(f"Error saving model: {e}")
                        no_improve = 0
                    else:
                        no_improve += 1
                        if no_improve >= patience:
                            print(f"Early stopping triggered after {epoch+1} epochs")
                            break

            except Exception as e:
                print(f"Error in epoch {epoch + 1}: {e}")
                # Attempt to save checkpoint even if epoch fails
                checkpoint_dict = {
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                }

                if batch_count > 0:
                    checkpoint_dict['train_loss'] = avg_train_loss
                if 'avg_val_loss' in locals():
                    checkpoint_dict['val_loss'] = avg_val_loss

                torch.save(checkpoint_dict, f'emergency_checkpoint_epoch_{epoch}.pth')
                continue

        end_time = time.time()
        training_time = end_time - start_time

    except Exception as e:
        print(f"Critical training error: {e}")
        raise
    finally:
        # Clean up
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    return train_losses, val_losses, rouge_scores, training_time


def plot_training_progress(train_losses, val_losses, rouge_scores):
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 3)
    for metric, scores in rouge_scores.items():
        plt.plot(scores, label=metric)
    plt.legend()
    plt.title('ROUGE Scores')

    plt.tight_layout()
    plt.savefig('training_progress.png')
    plt.close()
    files.download('training_progress.png')


# def generate_summary(model, article, tokenizer, device, max_length=128):
#     inputs = tokenizer(article, max_length=1024, truncation=True, return_tensors="pt").to(device)
#     summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
#     return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def generate_summary(model, article, tokenizer, device, max_length=None):
    """
    Generate a summary for the given article using the trained model.

    Args:
        model: The trained Pegasus model
        article: Input article text
        tokenizer: Pegasus tokenizer
        device: Device to run generation on
        max_length: Optional override for maximum length. If None, uses TRAINING_PARAMS value
    """
    # Use the same max length as training if not specified
    if max_length is None:
        max_length = TRAINING_PARAMS['MAX_TARGET_LENGTH']

    inputs = tokenizer(
        article,
        max_length=TRAINING_PARAMS['MAX_INPUT_LENGTH'],  # Use consistent input length
        truncation=True,
        return_tensors="pt"
    ).to(device)

    summary_ids = model.generate(
        inputs["input_ids"],
        num_beams=4,
        max_length=max_length,
        early_stopping=True,
        length_penalty=2.0,  # Added for better length control
        min_length=int(max_length/4),  # Added reasonable minimum length
        no_repeat_ngram_size=4  # Prevent repetition
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def inspect_frozen_params(model):
    frozen_params = []
    trainable_params = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            frozen_params.append(name)
        else:
            trainable_params.append(name)

    print("\nFrozen parameters:")
    for name in frozen_params:
        print(f"- {name}")

    print("\nNumber of frozen parameters:", len(frozen_params))
    print("Number of trainable parameters:", len(trainable_params))

def main():
    try:
        set_seed()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')
        config = create_small_pegasus_config()
        model = PegasusForConditionalGeneration(config).to(device)
        total_params,trainable_params = verify_model_size(model)
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")
        inspect_frozen_params(model)


        train_data, val_data = load_data(num_samples=15000)
        train_loader, val_loader = create_dataloaders(train_data, val_data, tokenizer, TRAINING_PARAMS['BATCH_SIZE'])
        train_losses, val_losses, rouge_scores, training_time = train_model(model, train_loader, val_loader, tokenizer, device, TRAINING_PARAMS['NUM_EPOCHS'])
        plot_training_progress(train_losses, val_losses,rouge_scores)

        print("Evaluating fine-tuned model...")
        fine_tuned_rouge = evaluate(model, val_loader, tokenizer, device)
        print("Fine-tuned Model Performance:")
        print(f"ROUGE Scores: {fine_tuned_rouge}")
        print(f"Total training time: {training_time:.2f} seconds")

        print("\nGenerating example summaries...")
        for i in range(3):
            article = val_data[i]["article"]
            reference = val_data[i]["highlights"]
            generated = generate_summary(model, article, tokenizer, device)
            print(f"\nArticle {i+1}:")
            print(f"Reference: {reference}")
            print(f"Generated: {generated}")
            print("-" * 50)
    finally:
        # Add cleanup at the end
        cleanup()

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total parameters: 108,582,400
Trainable parameters: 108,058,112

Frozen parameters:
- model.encoder.embed_positions.weight
- model.decoder.embed_positions.weight

Number of frozen parameters: 2
Number of trainable parameters: 341
Training samples: 13500
Validation samples: 1500
GPU Memory before epoch 1: 3452.28 MB


Epoch 1/20 [Val]: 100%|██████████| 375/375 [00:16<00:00, 22.58it/s]


Epoch 1/20
Average Train Loss: 10.2948
Average Val Loss: 8.3167
Learning Rate: 5.00e-06
Updated learning rate after plateau adjustment: 4.998518518518519e-06
Evaluating after epoch 1...


Evaluating: 100%|██████████| 375/375 [10:27<00:00,  1.67s/it]


Current ROUGE Scores: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}
New best model saved with validation loss: 8.3167
GPU Memory before epoch 2: 4689.84 MB


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluating fine-tuned model...


Evaluating: 100%|██████████| 375/375 [06:09<00:00,  1.02it/s]


Fine-tuned Model Performance:
ROUGE Scores: {'rouge1': 0.12260109995164857, 'rouge2': 0.014801708761302736, 'rougeL': 0.10146980858142782}
Total training time: 17975.81 seconds

Generating example summaries...

Article 1:
Reference: Zully Broussard decided to give a kidney to a stranger .
A new computer program helped her donation spur transplants for six kidney patients .
Generated: CNN.com: "I't know what you can't know that we't be a good," she says . She says it's no longer than a few years ago, he says . He says she's more than 200 percent of people have been a year .
--------------------------------------------------

Article 2:
Reference: The 20th MLS season begins this weekend .
League has changed dramatically since its inception in 1996 .
Some question whether rules regarding salary caps and transfers need to change .
Generated: South Africa's World Cup team has been held in the World Cup . The United States has been made in the world's first time since 2008 . The World Cup is